In [14]:
pip install pyspark

In [15]:
import sys
import re
from pyspark import SparkContext, SparkConf
import numpy as np

import time as time
from operator import itemgetter

In [16]:
# create Spark context with necessary configuration
sc = SparkContext("local","PySpark Word Count Exmaple")

In [18]:
filepath = 'email-Eu-core.txt'
links = sc.textFile(filepath)

#links.repartition(32)

#transpose
links_formatted = links.map(lambda x: (int(x.split(' ')[0]),int(x.split(' ')[1])))
links_formatted_t = links.map(lambda x: (int(x.split(' ')[1]),int(x.split(' ')[0])))


In [19]:
links_formatted_list = links_formatted.map(lambda x: (x[0],[x[1]]))
link_formatted_dict = links_formatted_list.reduceByKey(lambda a,b: a+b)

#transpose
links_formatted_list_t = links_formatted_t.map(lambda x: (x[0],[x[1]]))
link_formatted_dict_t = links_formatted_list_t.reduceByKey(lambda a,b: a+b)

print('The number of nodes in L is ',link_formatted_dict.count())
print('The number of nodes in L.T is ',link_formatted_dict_t.count())


The number of nodes in L is  868
The number of nodes in L.T is  991


In [20]:
hubs = links_formatted.flatMap(lambda x: x).distinct().map(lambda x: (x, 1.0))
n_nodes = hubs.count()

print('Total number of nodes is ', n_nodes)
elements = list(np.zeros(n_nodes))
rdd = sc.parallelize(elements)

rdd = rdd.zipWithIndex()

def getList(x):
  return_list = list(np.zeros(n_nodes))
  return return_list

matRdd = rdd.map(lambda x: getList(x[0]))

#transpose
matRdd_t = rdd.map(lambda x: getList(x[0]))


Total number of nodes is  1005


In [21]:
def getList_empty(x):
  return_list = []
  return return_list

empty_rdd = rdd.map(lambda x: getList_empty(x[0]))
empty_rdd = empty_rdd.zipWithIndex()
empty_rdd = empty_rdd.map(lambda x: (x[1], x[0]))

def add_two_lists(x):
  if (x[1][0]) == None :
    return (x[0],['A'])
  else:
    return (x[0],x[1][0])

zip_empty = link_formatted_dict.rightOuterJoin(empty_rdd).map(lambda x: add_two_lists(x))
zip_empty_p = zip_empty.repartition(1)

#transpose
zip_empty_t = link_formatted_dict_t.rightOuterJoin(empty_rdd).map(lambda x: add_two_lists(x))
zip_empty_p_t = zip_empty_t.repartition(1)


In [22]:
matRdd1 = matRdd.zipWithIndex()
matRdd_zip = matRdd1.map(lambda x: (x[1], x[0]))

#transpose
matRdd_t1 = matRdd_t.zipWithIndex()
matRdd_t_zip = matRdd_t1.map(lambda x: (x[1], x[0]))

sampleWordsRDD6 = zip_empty_p.join(matRdd_zip)
print(sampleWordsRDD6.count())

#transpose
sampleWordsRDD6_t = zip_empty_p_t.join(matRdd_t_zip)
print(sampleWordsRDD6_t.count())


1005
1005


In [23]:
def form_matrix(x):
  if x[1][0][0] == 'A':
    l1 = [0 for i in range(n_nodes)]
    return (x[0],l1)
  else:
    for k in x[1][0]:
      x[1][1][k] = 1
    return (x[0],x[1][1])

l_matrix = sampleWordsRDD6.map(lambda x: form_matrix(x))

#transpose
l_t_matrix = sampleWordsRDD6_t.map(lambda x: form_matrix(x))

In [24]:
hubs_ones = [1 for i in range(n_nodes)]
hubs = sc.parallelize(hubs_ones)
hubs_updated = hubs_ones

def product_sum(x):
  if x[1][1] == None:
    return 0
  else:
    return np.sum(np.array(x[1])*np.array(x[2]))

start0 = time.time()
for i in range(40):
  start = time.time()

  matrix_hubs = l_matrix.map(lambda x: (x[0],x[1],hubs_updated))
  matrix_product_hubs = matrix_hubs.map(lambda x: product_sum(x))

  auths_list = matrix_product_hubs.collect()
  auths_max = max(auths_list)
  auths_list = [auths_list[i]/auths_max for i in range(len(auths_list))]

  matrix_t_auths = l_t_matrix.map(lambda x: (x[0],x[1],auths_list))
  matrix_t_product_auths = matrix_t_auths.map(lambda x: product_sum(x))

  hubs_updated = matrix_t_product_auths.collect()
  hub_max = max(hubs_updated)
  hubs_updated = [hubs_updated[i]/hub_max for i in range(len(hubs_updated))]

  print(i, np.round(time.time() - start,4))

print('Total time : ', np.round(time.time() - start0,4))

0 1.7706
1 1.7311
2 1.5257
3 1.5631
4 1.8879
5 1.675
6 1.6494
7 1.6994
8 1.48
9 1.5338
10 1.5085
11 1.5353
12 1.464
13 1.5659
14 1.4853
15 1.5585
16 1.8655
17 1.5199
18 1.5641
19 1.4239
20 1.5484
21 1.4188
22 1.4626
23 1.4427
24 1.5297
25 1.4405
26 1.4632
27 1.4885
28 1.4373
29 1.3706
30 1.4453
31 1.3842
32 1.7776
33 1.4516
34 1.402
35 1.4797
36 1.3663
37 1.5991
38 1.4024
39 1.9217
Total time :  61.8838


In [25]:
tuple_hubs = []
tuple_auths = []

for i in range(0, n_nodes):
  t = (i, hubs_updated[i])
  tuple_hubs.append(t)

  t = (i, auths_list[i])
  tuple_auths.append(t)

sorted_hubs_asc = sorted(tuple_hubs,key=itemgetter(1), reverse=False)
sorted_hubs_dsc = sorted(tuple_hubs,key=itemgetter(1), reverse=True)

sorted_auths_asc = sorted(tuple_auths,key=itemgetter(1), reverse=False)
sorted_auths_dsc = sorted(tuple_auths,key=itemgetter(1), reverse=True)

In [26]:
print('Top nodes IDs with hub scores')
for i in range(5):
  print("{:<5} {:<5} {:<5} ".format(i+1, sorted_hubs_dsc[i][0], np.round(sorted_hubs_dsc[i][1],4)))

Top nodes IDs with hub scores
1     182   1.0   
2     28    0.8942 
3     543   0.8555 
4     31    0.8101 
5     547   0.7635 


In [27]:
print('Bottom nodes IDs with hub scores')
for i in range(5):
  print("{:<5} {:<5} {:<5} ".format(i+1, sorted_hubs_asc[i][0], np.round(sorted_hubs_asc[i][1],4)))

Bottom nodes IDs with hub scores
1     289   0.0   
2     407   0.0   
3     464   0.0   
4     488   0.0   
5     491   0.0   


In [28]:
print('Top nodes IDs with authority scores')
for i in range(5):
  print("{:<5} {:<5} {:<5} ".format(i+1, sorted_auths_dsc[i][0], np.round(sorted_auths_dsc[i][1],4)))

Top nodes IDs with authority scores
1     40    1.0   
2     17    0.7598 
3     656   0.7553 
4     99    0.7333 
5     629   0.6497 


In [29]:
print('Bottom nodes IDs with authority scores')
for i in range(5):
  print("{:<5} {:<5} {:<5} ".format(i+1, sorted_auths_asc[i][0], np.round(sorted_auths_asc[i][1],4)))

Bottom nodes IDs with authority scores
1     429   0.0   
2     430   0.0   
3     431   0.0   
4     432   0.0   
5     433   0.0   
